In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import scikitplot as skplt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import KFold

# Carregando base de dados  pré-processada

In [ ]:
dataset = pd.read_csv("../../Database/dataBaseWithNER.csv")
dataset = dataset.drop(columns=["Unnamed: 0"])
dataset = dataset.dropna()
targets = np.array(dataset["target"].array)

dataset

In [ ]:
emailsText = []
for email in dataset["email"]:
    emailsText.append(email)

del dataset

# Representação vetorial Bag of Words

In [ ]:
vectorizer = CountVectorizer(max_features=2100)
X = vectorizer.fit_transform(emailsText)

bag = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())

del emailsText
del X

bag

In [ ]:
bag = np.array(bag)
bag

# Visualização de dados com TSNE

In [ ]:
# model = TSNE(n_components=2, random_state=0)
# #model = PCA(n_components=50, svd_solver='full')
# array_red = model.fit_transform(bag)
#
# df_tsne = pd.DataFrame(array_red)
#
# df_tsne['Target'] = target
# df_tsne_c1 = df_tsne[df_tsne['Target'] == 0]
#
# df_tsne_c2 = df_tsne[df_tsne['Target'] == 1]
#
# plt.scatter(df_tsne_c1[0].array,df_tsne_c1[1].array,marker='o',color='blue')
#
# plt.scatter(df_tsne_c2[0].array,df_tsne_c2[1].array,marker='o',color='red')
#
# plt.title('Dados')
# plt.xlabel('x')
# plt.ylabel('y')
#
# plt.show()

# Validação

In [ ]:
foldsAccuracy = []
foldLosses = []

In [ ]:

kfold = KFold(n_splits=4, shuffle=True)

In [ ]:
foldCount = 1
for train, test in kfold.split(bag, targets):
    model = keras.models.Sequential([
        ########## MLP
        keras.layers.Flatten(input_shape=(bag.shape[1],)),
        keras.layers.Dense(1000, activation="relu"),
        keras.layers.Dense(1000, activation="relu"),
        keras.layers.Dense(1000, activation="relu"),

        keras.layers.Dense(len(set(targets)), activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

    print('****************************************************')
    print(f'Iniciando treinamento da fold: {foldCount}.')

    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4,mode='min'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, restore_best_weights=True)]

    history = model.fit(bag[train], targets[train], epochs=200, callbacks=callbacks, validation_split=0.05)

    scores = model.evaluate(bag[test], targets[test], verbose=0)
    print(f'Score fold {foldCount}: {model.metrics_names[0]} de {scores[0]}; {model.metrics_names[1]} de {scores[1]*100}%')

    foldsAccuracy.append(scores[1] * 100)
    foldLosses.append(scores[0])

    foldCount = foldCount + 1

In [ ]:
print('****************************************************')
print('Score de cada fold:')
for i in range(0, len(foldsAccuracy)):
    print('****************************************************')
    print(f'--> Fold {i+1}: Loss: {foldLosses[i]} ; Accuracy: {foldsAccuracy[i]}%')

print('****************************************************')
print('Média de accuracy das folds:')
print(f'--> Accuracy: {np.mean(foldsAccuracy)} (+- {np.std(foldsAccuracy)})')
print(f'--> Loss: {np.mean(foldLosses)}')
print('****************************************************')

In [ ]:
#cm = confusion_matrix(testTarget,np.argmax(model.predict(bagTest.values), axis=-1))
#
#labels = ["Ham", "Spam"]
#
#cm_df = pd.DataFrame(cm, columns=labels)
#
#
#fig, ax = plt.subplots(figsize=(10,10))
#sns.heatmap(cm_df, annot=True, fmt="d", xticklabels=labels, yticklabels=labels)
#
#plt.show()